In [146]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

import string
import re
from keras.preprocessing.text import Tokenizer

In [147]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=10000)
# for human-friendly printing
labels = ['fake', 'real']

# read in our saved dictionary
with open('./Dictionary_Models/glove_models_dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [148]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
#     words = pad_sequences(words, padding='post', maxlen=23)
#     print(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
    return wordIndices

In [149]:
json_file = open('./NN_Models/Glove_BLSTM_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

model.load_weights('./NN_Models/Glove_BLSTM_model.h5')


In [150]:
model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 100, 50)           769850    
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 100, 50)           0         
_________________________________________________________________
bidirectional_8 (Bidirection (None, 100, 256)          183296    
_________________________________________________________________
bidirectional_9 (Bidirection (None, 128)               164352    
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 129       
Total params: 1,117,627
Trainable params: 347,777
Non-trainable params: 769,850
_______________________________________

In [152]:
tweetText = []
tweetText.append('there is a fire in the mountain top and nobody seems to be on the run')
tweetText.append('there is an accident happening on elim street')
tweetText.append('the cops shot a kid')
tweetText.append('the boys were playing soccer with eden hazard to day')

y = []
y.append(1)
y.append(1)
y.append(1)
y.append(0)

input = []
# testArr = convert_text_to_index_array(tweetText)
for i,k in enumerate(y):
#     y[i] = convert_text_to_index_array(y[i])
    tweetText[i] = convert_text_to_index_array(tweetText[i])

input = pad_sequences(tweetText, padding='post', maxlen=maxlen)
    

# model.compile(optimizer='adam',
#               loss='binary_crossentropy',
#               metrics=['accuracy','Precision','Recall'])
# y = np.array(y)
pred = model.predict_generator(input)

def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1
    
predictions_final = [rounding(x) for x in pred]

print(np.array(pred))

# print('\n',tweetText)

print('****',labels[np.argmax(pred)], 'Tweet')


[[0.3606582 ]
 [0.8805345 ]
 [0.3035707 ]
 [0.07187507]]
**** real Tweet
